In [ ]:
import pandas as pd
import ast
from pandas import json_normalize

In [ ]:
df_movies=pd.read_csv('C:/Users/NOVATEC/Desktop/movies_dataset.csv', sep=',', encoding='utf_8')

In [ ]:
#Vamos a guardar toda la información en un DataFrame para poder aceder a él una vez
#que esté desanidado

df_belongs_to_colection_copy=df_movies['belongs_to_collection']
df_belongs_to_colection=pd.DataFrame(df_belongs_to_colection_copy)

In [ ]:
#Convertimos todos los valores a listas de diccionarios para mejor manejo y poder tratar valores nulos

def evaluation(v):
    try:
        return ast.literal_eval(v)
    except (ValueError, SyntaxError):
        return None

df_belongs_to_colection['belongs_to_collection']= df_belongs_to_colection['belongs_to_collection'].apply(lambda x: evaluation(x) if pd.notnull(x) else None)


In [ ]:
#Normalizamos el DataFrame 

df_belongs= json_normalize(df_belongs_to_colection['belongs_to_collection'])

df_belongs['id'].fillna(0, inplace=True)
df_belongs['id']=df_belongs['id'].astype(int)

df_belongs=df_belongs.rename(columns={'id':'id_belongs', 'name':'collection_name', 'poster_path':'collection_poster_path', 'backdrop_path':'collection_backdrop_path'})

In [ ]:
#Guardamos la columna "production_companies" y lo convertimos a DataFrame


df_pc=df_movies['production_companies']
df_pc2=pd.DataFrame(df_pc)

#Después, aplicamos la misma función para manejar listas de diccionarios y valores nulos

df_pc2['production_companies']=df_pc2['production_companies'].apply(lambda x: evaluation(x) if pd.notnull(x) else None)

#Expandimos las listas 

df_pc3=df_pc2.explode('production_companies')

#Normalizamos los datos
df_production_companies=json_normalize(df_pc3['production_companies'])

#Convertimos a int los id

df_production_companies['id'].fillna(0, inplace=True)
df_production_companies['id']=df_production_companies['id'].astype(int)
df_production_companies=df_production_companies.rename(columns={'name':'company_name','id':'company_id'})

In [ ]:
#Aplicamos el mismo proceso para la columna "production_countries"

df_c=df_movies['production_countries']
df_c1=pd.DataFrame(df_c)
df_c1['production_countries']=df_c1['production_countries'].apply(lambda x: evaluation(x) if pd.notnull(x) else None)
df_c2=df_c1.explode('production_countries')
df_production_countries=json_normalize(df_c2['production_countries'])
df_production_countries=df_production_countries.rename(columns={'iso_3166_1':'country_id','name':'country'})

In [ ]:
#Aplicamos el mismo proceso para la columna "spoken_languages"

df_m=df_movies['spoken_languages']
df_m1=pd.DataFrame(df_m)
df_m1['spoken_languages']=df_m1['spoken_languages'].apply(lambda x: evaluation(x) if pd.notnull(x) else None)
df_m2=df_m1.explode('spoken_languages')
df_spoken_languages=json_normalize(df_m2['spoken_languages'])
df_spoken_languages=df_spoken_languages.rename(columns={'iso_639_1':'language_id','name':'language'})

In [ ]:
#Rellenamos con 0 los campos nulos de las columnas "revenue" y "budget"

df_movies['revenue'].fillna(0, inplace=True)
df_movies['budget'].fillna(0, inplace=True)

In [ ]:
df_g=df_movies['genres']
df_g1=pd.DataFrame(df_g)
df_g1['genres']=df_g1['genres'].apply(lambda x: evaluation(x) if pd.notnull(x) else None)
df_g2=df_g1.explode('genres')
df_genres=json_normalize(df_g2['genres'])
df_genres['id'].fillna(0, inplace=True)
df_genres['id']=df_genres['id'].astype(int)
df_genres=df_genres.rename(columns={'id':'genre_id','name':'genre_name'})

In [ ]:
#Concatemos los DataFrames creados con el original sin quitar nada de valores nulos para
#no causar inconsistencia en las filas

df_movies2=pd.concat([df_movies, df_belongs, df_production_companies, df_production_countries, df_spoken_languages, df_genres], axis=1)

In [ ]:
#Eliminamos las filas con valores nulos de "release_date"

df_movies2=df_movies2.dropna(subset=['release_date'])

In [ ]:
#Convertimos los valores de la columns "release_date" a formato "aaaa-mm-dd"

df_movies2['release_date']=pd.to_datetime(df_movies2['release_date'], errors='coerce')

In [ ]:
#Agregamos una nueva columna "release_year" con el año de "release_date"
#para después darle formato de "year"

df_movies2['release_year']=df_movies2['release_date'].dt.year.astype('Int64')

In [ ]:
#La columna "budget" tiene strings en ciertas filas, por lo que necesitamos trabajar con ellos
#esta función permite trabajar con ellos y asi darle formato float a los números

def conversion(v):
    try:
        return float(v)
    except ValueError:
        return v

df_movies2['budget'] = df_movies2['budget'].apply(conversion)

In [ ]:
#La siguiente función nos permite cambiar los datos de "budget" a float
#porque algunas filas dentro de ella contienen palabras

import numpy as np
def conversion(v):
    try:
        return float(v)
    except ValueError:
        return np.nan  # Usar np.nan para valores que no se pueden convertir


df_movies2['budget'] = df_movies2['budget'].apply(conversion)

In [ ]:
#Función que nos permite evitar el error de que el resultado de 0

def retn(r, b):
    if b == 0:
        return 0  
    else:
        return (r/b)
    
df_movies2['return'] = df_movies2.apply(lambda row: retn(row['revenue'], row['budget']), axis=1)

In [ ]:
#Eliminamos las columnas que no vamos a utilizar

df_movies2=df_movies2.drop(columns=['belongs_to_collection', 'genres', 'production_companies',
                                    'production_countries', 'spoken_languages', 'video',
                                    'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

In [ ]:
#Borramos las filas que tengan "vote count" de cero para tener un menor número de datos

df_movies2=df_movies2[df_movies2['vote_count']!=0]

In [ ]:
#Borramos las filas que tengas "vote_average" de cero, y con esto logramos tener películas
#que a gente haya visto y calificado

df_movies2=df_movies2[df_movies2['vote_average']!=0]

In [ ]:
#Eliminamos las películas que estén en los siguientes idiomas:

l=['English', 'Français', 'Español', 'Deutsch']

#Esto con el fin de seguir filtrando y tener un DataFrame más limpio

df_movies2=df_movies2[df_movies2['language'].isin(l)]

In [ ]:
#Ahora vamos a trabajar con el csv "credits"

df_credits=pd.read_csv('C:/Users/NOVATEC/Desktop/credits.csv', sep=',', encoding='utf-8')

In [ ]:
#Desanidamos los datos de la columna "cast"

df_ct1=df_credits['cast'].astype(str)
df_ct2=df_ct1.apply(ast.literal_eval)
df_cast=df_ct2.explode().apply(pd.Series)

In [ ]:
df_cast=df_cast.rename(columns={'id':'name_id','name':'actor_name'})

In [ ]:
dlt=df_cast.columns[8]
df_cast=df_cast.drop(columns=dlt)

In [ ]:
#Desanidamos los datos de la columna crew

df_cw=df_credits['crew']
df_cw2=df_cw.apply(ast.literal_eval)
df_crew=df_cw2.explode().apply(pd.Series)

In [ ]:
df_crew['id'].fillna(0, inplace=True)
df_crew['id']=df_crew['id'].astype(int)

In [ ]:
df_crew=df_crew.rename(columns={'gender':'creew_gender','id':'crew_id','name':'crew_name'})

In [ ]:
dlt2=df_crew.columns[7]
df_crew=df_crew.drop(columns=dlt2)

In [ ]:
#Quitamos columnas y filtramos para tener un DataFrame más limpio

df_cast=df_cast.drop(columns=['profile_path'])
df_cast=df_cast[df_cast['order']!=0]
df_cast=df_cast.dropna(subset=['actor_name'])


In [ ]:
#De la misma manera con df_crew

df_crew=df_crew.dropna(subset=['crew_name'])
df_crew=df_crew.drop(columns=['profile_path'])
df_crew

In [ ]:
#creamos un diccionario con los meses

months={
    'enero': 1,
    'febrero': 2,
    'marzo': 3,
    'abril': 4,
    'mayo': 5,
    'junio': 6,
    'julio': 7,
    'agosto': 8,
    'septiembre': 9,
    'octubre': 10,
    'noviembre': 11,
    'diciembre': 12
}

def cantidad_filmaciones_mes(df_movies2, m):
    # Obtenemos el número del mes

    month_number=months.get(m.lower())
    if month_number is None:
        return f"El mes '{m}' no es válido."
    
    # Filtramos las películas por mes
    movies_per_month=df_movies2[df_movies2['release_date'].dt.month==month_number]
    
    # Mostramos nombres y fechas de las películas una vez filtradas
    for index, row in movies_per_month.iterrows():
        print(f"Nombre: {row['title']}, Fecha de estreno: {row['release_date'].date()}")
    
    q=len(movies_per_month)
    
    return f"{q} Cantidad de peliculas fueron estrenadas el mes de: {m}"

In [ ]:
#Creamos un diccionario con los días de la semana

days_week={
    'lunes': 0,
    'martes': 1,
    'miércoles': 2,
    'jueves': 3,
    'viernes': 4,
    'sábado': 5,
    'domingo': 6
}

def cantidad_filmaciones_dia(df_movies2, d):
    #Obtenemos el número del día de la semana


    day_number=days_week.get(d.lower())
    if day_number is None:
        return f"El día '{d}' no es válido."
    
    #Filtramos las películas por día de semana
    movies_per_day=df_movies2[df_movies2['release_date'].dt.dayofweek==day_number]
    
    #Mostramos nombres y fechas de las películas una vez filtradas
    for index, row in movies_per_day.iterrows():
        print(f"Nombre: {row['title']}, Fecha de estreno: {row['release_date'].date()}")
    
    q=len(movies_per_day)
    
    return f"{q} Cantidad de peliculas fueron estrenadas el día: {d}"

In [ ]:
def score_titulo(df_movies2, t):
    #Filtramos por título de película

    movie=df_movies2[df_movies2['title']==t]
    
    if movie.empty:
        return f"La película '{t}' no se encuentra en el DataFrame."
    
    #Extraemos la información de la película
    rel_date=movie['release_date'].iloc[0]
    popty=movie['popularity'].iloc[0]
    rel_year=rel_date.year
    
    
    return {f" La pelicula '{t}' fue estrenada en el año {rel_year} con una popularidad de {popty}"}

In [ ]:
def votos_titulo(df_movies2, tl):
    #Filtrar título de película

    mo=df_movies2[df_movies2['title']==tl]
    
    if mo.empty:
        return f"La película '{tl}' no se encuentra en el DataFrame."
    
    #Verificamos que la cantidad de votos es mayor a 2000
    if mo['vote_count'].iloc[0]>2000:
        i=mo[['vote_count', 'vote_average', 'release_date']].iloc[0]
        return i
    else:
        return f"La película '{tl}' no cumple con la condición de tener más de 2000 votos."


In [ ]:
df_movies2['id']=df_movies2['id'].astype(str)
df_cast['cast_id']=df_cast['cast_id'].astype(str)

def get_actor(df_movies2, df_cast, actor):
    #Unimos por la columna 'id' de df_movies2 y 'cast_id' de df_cast
    df_joined=df_cast.merge(df_movies2, left_on='cast_id', right_on='id')
    
    #Filtrar por nombre de actor
    movies_actor=df_joined[df_joined['actor_name']==actor]
    
    if movies_actor.empty:
        return f"No se encontraron películas para el actor '{actor}'."
    
    #Para calcular el total generado
    total_return=movies_actor['return'].sum()
    
    #Para contar el número de películas
    numer_of_movies=movies_actor.shape[0]
    
    #Para calcular el promedio de 'return'
    average_return=movies_actor['return'].mean()
    
    
    return f"El actor '{actor}' ha generado un total de {total_return:.2f} en 'return', ha participado en {numer_of_movies} películas, con un promedio de 'return' de {average_return:.2f}."

In [ ]:
def get_director(df_crew, df_movies2, director):
    #Filtramos por directores y que solo tenga el "job" de DIRECTOR
    df_crew=df_crew[(df_crew['job']=='director')&(df_crew['crew_name']==director)]

    #Realizar un merge con de df_movies2 y df_crew
    df_merged=pd.merge(df_crew, df_movies2, left_on='credit_id', right_on='id', how='inner')

    #Para calcular la cantidad de películas
    num_movies=df_merged.shape[0]

    #Para calcular el retorno total
    total_return=df_merged['return'].sum()

    #Para calcular el retorno promedio
    average_return=df_merged['return'].mean() if num_movies>0 else 0

    return num_movies, total_return, average_return